In [87]:
import re
import bs4
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Himank
[nltk_data]     K\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('all_kindle_review.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
df = df[['reviewText', 'rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [4]:
df.shape

(12000, 2)

In [5]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [6]:
df['rating'].value_counts()

5    3000
4    3000
3    2000
2    2000
1    2000
Name: rating, dtype: int64

In [7]:
df['rating'] = df['rating'].apply(lambda x: 0 if x<3 else 1)
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [8]:
df['rating'].value_counts()

1    8000
0    4000
Name: rating, dtype: int64

In [18]:
df['reviewText'] = df['reviewText'].str.lower()
df.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read i want put i read one sitting...,1
2,i start saying first four books i expecting 34...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,i expect type book library pleased find price ...,1


In [11]:
df

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,1
11996,I have read all seven books in this series. Ap...,1
11997,This book really just wasn't my cuppa. The si...,1
11998,"tried to use it to charge my kindle, it didn't...",0


In [12]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

KeyboardInterrupt: 

In [21]:
df.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read i want put i read one sitting...,1
2,i start saying first four books i expecting 34...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,i expect type book library pleased find price ...,1


In [23]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

df['reviewText'] = df['reviewText'].apply(lambda x: lemmatize_words(x))

In [24]:
df.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read i want put i read one sitting...,1
2,i start saying first four book i expecting 34 ...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,i expect type book library pleased find price ...,1


In [141]:
X = df['reviewText'].values
y = df['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [143]:
words = [i.split() for i in X_train]

In [144]:
model = Word2Vec(words)

In [145]:
model.corpus_count, model.epochs

(9600, 5)

In [146]:
def avg_word2vec(doc):
    valid_words = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    return np.mean(valid_words, axis=0) if valid_words else np.zeros(model.vector_size)

In [148]:
def vectorize(column):
    col_vec = [i.split() for i in column]
    for i in range(len(col_vec)):
        col_vec[i] = avg_word2vec(col_vec[i])
    return np.array(col_vec)

In [149]:
X_train_vec = vectorize(X_train)
X_test_vec = vectorize(X_test)

In [154]:
classifier = RandomForestClassifier()
classifier.fit(X_train_vec, y_train)

RandomForestClassifier()

In [155]:
y_pred = classifier.predict(X_test_vec)

In [156]:
accuracy_score(y_test, y_pred)

0.7595833333333334

In [157]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.54      0.60       803
           1       0.79      0.87      0.83      1597

    accuracy                           0.76      2400
   macro avg       0.73      0.70      0.71      2400
weighted avg       0.75      0.76      0.75      2400



In [163]:
classifier.predict(vectorize(['asdlfkjasdlfjas']))

array([0], dtype=int64)

In [16]:
df = pd.read_csv('cleaned_data_0.csv')